# RecursiveCharacterTextSplitter로 문서 분할하기

이 노트북에서는 **RecursiveCharacterTextSplitter**를 사용하여 문서를 적절한 크기의 청크로 분할하는 방법을 알아봅니다.

## RAG 파이프라인에서의 위치

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  📄 Load    │ ──▶ │  ✂️ Split   │ ──▶ │  🔢 Embed   │ ──▶ │  💾 Store   │
│  (로드)     │     │  (분할)     │     │  (임베딩)    │     │  (저장)     │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                          ↑
                       현재 단계
```

## 왜 문서를 분할해야 할까?

1. **LLM 컨텍스트 제한**: 모델의 최대 토큰 수 제한
2. **검색 정확도**: 작은 청크가 더 정확한 검색 결과
3. **비용 효율**: 필요한 부분만 LLM에 전달

## RecursiveCharacterTextSplitter란?

**재귀적**으로 여러 구분자를 시도하여 의미 있는 단위로 텍스트를 분할합니다.

기본 구분자 순서:
1. `\n\n` (단락)
2. `\n` (줄바꿈)
3. ` ` (공백)
4. `` (문자)

---

# 1. 패키지 설치

In [ ]:
!pip install -q langchain langchain-community langchain-text-splitters

# 2. 테스트 파일 준비

In [ ]:
# 테스트용 텍스트 파일 생성
sample_text = '''LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다.

주요 기능:
1. 프롬프트 관리 및 최적화
2. 체인 구성을 통한 복잡한 워크플로우 생성
3. 다양한 데이터 소스와의 연동
4. 메모리 관리를 통한 대화 컨텍스트 유지

LangChain을 사용하면 RAG(Retrieval-Augmented Generation) 시스템을 쉽게 구축할 수 있습니다. RAG는 외부 지식을 검색하여 LLM의 응답을 향상시키는 기술입니다.

Vector Store는 임베딩된 문서를 저장하고 유사도 검색을 수행하는 데이터베이스입니다. 대표적으로 Chroma, Pinecone, Weaviate 등이 있습니다.
'''

with open('./test.txt', 'w', encoding='utf-8') as f:
    f.write(sample_text)

print(f"test.txt 생성 완료 (총 {len(sample_text)}자)")

# 3. 문서 로드 및 분할

**핵심 파라미터:**

```python
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # 청크 최대 크기 (문자 수)
    chunk_overlap=200   # 청크 간 중복 (문맥 유지)
)
```

| 파라미터 | 설명 |
|----------|------|
| `chunk_size` | 각 청크의 최대 문자 수 |
| `chunk_overlap` | 인접 청크 간 겹치는 문자 수 (문맥 유지용) |

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 문서 로드
loader = TextLoader('./test.txt', encoding='utf-8')
docs = loader.load()

print(f"원본 문서 수: {len(docs)}")
print(f"원본 문서 길이: {len(docs[0].page_content)}자")

In [ ]:
# 2. Text Splitter 생성
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,     # 예시를 위해 작은 값 사용
    chunk_overlap=50
)

# 3. 문서 분할
splitted_docs = splitter.split_documents(docs)

print(f"분할된 문서 수: {len(splitted_docs)}")

# 4. 분할 결과 확인

In [ ]:
# 각 청크 확인
for i, doc in enumerate(splitted_docs):
    print(f"\n=== 청크 {i+1} ({len(doc.page_content)}자) ===")
    print(doc.page_content)
    print(f"metadata: {doc.metadata}")

---

## chunk_overlap의 역할

```
청크 1: [    텍스트 A    ][중복]
청크 2:              [중복][    텍스트 B    ]
```

- 문맥이 청크 경계에서 끊기는 것을 방지
- 검색 시 관련 정보가 누락되지 않도록 보장

## 적절한 chunk_size 선택

| chunk_size | 장점 | 단점 |
|------------|------|------|
| 작음 (200-500) | 정확한 검색 | 문맥 손실 가능 |
| 중간 (500-1000) | 균형 잡힘 | - |
| 큼 (1000-2000) | 풍부한 문맥 | 검색 정확도 저하 |

## 다른 Text Splitter

```python
# 토큰 기반 분할
from langchain_text_splitters import TokenTextSplitter
splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50)

# 문장 기반 분할
from langchain_text_splitters import SentenceTransformersTokenTextSplitter
```